# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察填補缺值以及 標準化 / 最小最大化 對數值的影響

# [作業重點]
- 觀察替換不同補缺方式, 對於特徵的影響 (In[4]~In[6], Out[4]~Out[6])
- 觀察替換不同特徵縮放方式, 對於特徵的影響 (In[7]~In[8], Out[7]~Out[8])

In [6]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df_test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
df_train.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [25]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns): #依照序列zip成數組
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature) #numerical features
print(f'{len(num_features)} Numeric Features : {num_features}\n') #f'輸出

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



In [26]:
# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
train_num = train_Y.shape[0]#樣本人數 samples
df.head()

,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


# 作業1
* 試著在補空值區塊, 替換並執行兩種以上填補的缺值, 看看何者比較好?

In [37]:
# 空值補 -1, 做羅吉斯迴歸
df_m1 = df.fillna(-1) # numierical dataframe 填補成-1
train_X = df_m1[:train_num]#全部的樣本
estimator1 = LogisticRegression(solver='lbfgs')
option1 = cross_val_score(estimator1,train_X, train_Y, cv=5).mean()

option1

0.6982644788418415

In [35]:


df_zero = df.fillna(0)
train_X2 = df_zero[:train_num]
estimator2 = LogisticRegression(solver='lbfgs')
cv_5 = cross_val_score(estimator2 , train_X2 , train_Y , cv=5) #會產生五個值的array
option2 = cv_5.mean() 

option2

0.6993817972775958

# 作業2
* 使用不同的標準化方式 ( 原值 / 最小最大化 / 標準化 )，搭配羅吉斯迴歸模型，何者效果最好?

In [38]:
"""
Your Code Here
"""
Scores = []
Scores.extend([option1,option2])

std_scaler = StandardScaler()
std_train_X = std_scaler.fit_transform(train_X) #numpy array of shape [n_samples, n_features]
estimator = LogisticRegression(solver='lbfgs')
score1 = cross_val_score(estimator , std_train_X , train_Y,cv=5).mean()

Scores.append(score1)

minmax_scaler = MinMaxScaler()
minmax_train_X = minmax_scaler.fit_transform(train_X)
estimator = LogisticRegression(solver='lbfgs')
score2 = cross_val_score(estimator , minmax_train_X , train_Y , cv=5).mean()


Scores.append(score2)

In [39]:
Scores

[0.6982644788418415,
 0.6993817972775958,
 0.6982582017719778,
 0.7004864906462793]

In [41]:
train_X.head(10) # Dataframe -> multi-dimensional-array

,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500
5,3,-1.0,0,0,8.4583
6,1,54.0,0,0,51.8625
7,3,2.0,3,1,21.0750
8,3,27.0,0,2,11.1333
9,2,14.0,1,0,30.0708


In [40]:
std_train_X

array([[ 0.82737724, -0.0896342 ,  0.43279337, -0.47367361, -0.50244517],
       [-1.56610693,  0.80634954,  0.43279337, -0.47367361,  0.78684529],
       [ 0.82737724,  0.13436174, -0.4745452 , -0.47367361, -0.48885426],
       ...,
       [ 0.82737724, -1.37761082,  0.43279337,  2.00893337, -0.17626324],
       [-1.56610693,  0.13436174, -0.4745452 , -0.47367361, -0.04438104],
       [ 0.82737724,  0.47035564, -0.4745452 , -0.47367361, -0.49237783]])